## ROBO QUE DESCOMPACTA OS ARQUIVOS .ZIP

In [ ]:

import os
import zipfile
import pandas as pd
from pathlib import Path


caminho = Path(os.getcwd())

data_folder = caminho / "Data"

arquivos_escolas = []
arquivos_perfil_educandos = []

for pasta in data_folder.iterdir():
    if pasta.is_dir():
        if pasta.name == "Escolas":
            for arquivo in pasta.iterdir():
                if arquivo.name.endswith(".csv") and not arquivo.name.startswith("dicionario"):
                    df = pd.read_csv(arquivo, encoding="latin1", sep=";")
                    arquivos_escolas.append(df)
        elif pasta.name == "Perfil dos educandos":
            for arquivo in pasta.iterdir():
                if arquivo.name.endswith("csv") and not arquivo.name.startswith("dicionario"):
                    df = pd.read_csv(arquivo, encoding="latin1", sep=";")
                    arquivos_perfil_educandos.append(df)
        else:
            print("Esta iteração não retornou pastas válidas")

df_escolas = pd.concat(arquivos_escolas, ignore_index=True)
df_perfil_educandos = pd.concat(arquivos_perfil_educandos, ignore_index=True)
df_escolas.to_excel("escolas.xlsx", index=False)
df_perfil_educandos = df_perfil_educandos.sample(n=1000000)
df_perfil_educandos.to_excel("perfil_educandos.xlsx", index=False)

## TRATAMENTO DA BASE ESCOLAS

In [8]:
import pandas as pd
import os

file_path = "escolas.xlsx" 
df = pd.read_excel(file_path, sheet_name="Sheet1")

columns_to_keep = df.columns[:34] 
df_selected = df[columns_to_keep]

df_selected = df_selected.dropna(how='all')

df_selected = df_selected[df_selected.iloc[:, 0].notna()]

df_selected.reset_index(drop=True, inplace=True)

output_file = "escolas_tratadas.xlsx"


if os.path.exists(output_file):
    output_file = "escolas_tratadas_novo.xlsx"


df_selected.to_excel(output_file, index=False)

print(f"Base tratada e salva como '{output_file}'")

Base tratada e salva como 'escolas_tratadas_novo.xlsx'


## CONECTANDO AS BASES NO SQLITE

In [9]:
import sqlite3
import pandas as pd

conexao = sqlite3.connect("Case_USE.db")

cursor = conexao.cursor()

print('lendo df_escolas')
df_escolas = pd.read_excel("escolas_tratadas.xlsx")
print('lendo df_educandos')
df_educandos = pd.read_excel("educandos_reduzida.xlsx")


lendo df_escolas
lendo df_educandos


## CONEXÃO BANCO DE DADOS

In [10]:
# Conectar ao banco de dados SQLite
conn = sqlite3.connect("Case_USE.db")
cursor = conn.cursor()

# Criar a tabela com base nas colunas do Excel
colunas_escolas = ", ".join([f"{col} TEXT" for col in df_escolas.columns])
colunas_educandos = ", ".join([f"{col} TEXT" for col in df_educandos.columns])

In [11]:
import pandas as pd
df = pd.read_excel("educandos_reduzida.xlsx")
df.columns

Index(['DRE', 'CODESC', 'TIPOESC', 'NOMESC', 'DISTRITO', 'SETEDU', 'ANO',
       'REDE', 'MODAL', 'MODALID', 'SERIEV', 'DESCSERIE', 'PERIODO', 'TURNO',
       'DESCTURNO', 'SEXO', 'IDADE', 'NEE', 'RACA', 'QTDE', 'DATABASE',
       'ï»¿dre', 'codes', 'rede', 'modal', 'descserie', 'periodo', 'turno',
       'descturno', 'sexo', 'idade', 'nee', 'raca', 'qtd', 'database', 'dre',
       'SETOR', 'Qtde'],
      dtype='object')

## TRATAMENTO BASE PERFIL DOS EDUCANDOS

In [12]:
df = pd.read_excel('educandos_reduzida.xlsx')
df.drop(['MODALID','SERIEV'],axis=1)

colunas = df.columns

df = pd.DataFrame(columns=colunas)


df.columns = df.columns.str.lower()  # Convertendo para minúsculas
df.columns = df.columns.str.replace('ï»¿dre', 'dre2')  # Removendo caracteres especiais


colunas_para_unificar = {
    'DRE': ['DRE','dre', 'dre2' ],
    'CODESC': ['CODESC', 'codes'],
    'TIPOESC': ['TIPOESC', 'tipoesc'],
    'NOMESC': ['NOMESC','nomesc'],
    'DISTRITO': ['DISTRITO', 'tiposc'],
    'SETEDU': ['SETEDU', 'setor'],
    'ANO': ['ANO','ano'],
    'REDE': ['REDE','rede'],
    'MODAL': ['MODAL', 'modal'],
    'modalid': ['modalid'],
    'seriev': ['seriev'],
    'DESCSERIE': ['DESCSERIE','descserie'],
    'PERIODO': ['PERIODO','periodo'],
    'TURNO': ['TURNO','turno'],
    'DESCTURNO': ['DESCTURNO', 'descturno'],
    'SEXO': ['SEXO','sexo'],
    'IDADE': ['IDADE', 'idade'],
    'NEE': ['NEE','nee'],
    'RACA': ['RACA','raca'],
    'QTDE': ['QTDE', 'qtd', 'Qtde'],
    'DATABASE': ['DATABASE','database']
}

In [ ]:
df = pd.read_excel('educandos_reduzida.xlsx')

df['QTDE'] = df[['QTDE', 'qtd', 'Qtde']].sum(axis=1, skipna=True)


df['DRE'] = df[['DRE', 'dre']].bfill(axis=1).iloc[:, 0]  # Preencher de trás pra frente
df['NOMESC'] = df[['NOMESC', 'nomesc']].bfill(axis=1).iloc[:, 0]
df['DISTRITO'] = df[['DISTRITO', 'distrito']].bfill(axis=1).iloc[:, 0]
df['SETOR'] = df[['SETOR', 'setor']].bfill(axis=1).iloc[:, 0]
df['SETOR'] = df[['SETOR', 'SETEDU']].bfill(axis=1).iloc[:, 0]
df['ANO'] = df[['ANO', 'ano']].bfill(axis=1).iloc[:, 0]
df['REDE'] = df[['REDE', 'rede']].bfill(axis=1).iloc[:, 0]
df['MODAL'] = df[['MODAL', 'modal']].bfill(axis=1).iloc[:, 0]
df['MODAL'] = df[['MODAL', 'MODALID']].bfill(axis=1).iloc[:, 0]
df['DESCSERIE'] = df[['DESCSERIE', 'descserie']].bfill(axis=1).iloc[:, 0]
df['DESCSERIE'] = df[['DESCSERIE', 'SERIEV']].bfill(axis=1).iloc[:, 0]
df['PERIODO'] = df[['PERIODO', 'periodo']].bfill(axis=1).iloc[:, 0]
df['TURNO'] = df[['TURNO', 'turno']].bfill(axis=1).iloc[:, 0]
df['SEXO'] = df[['SEXO', 'sexo']].bfill(axis=1).iloc[:, 0]
df['IDADE'] = df[['IDADE', 'idade']].bfill(axis=1).iloc[:, 0]
df['NEE'] = df[['NEE', 'nee']].bfill(axis=1).iloc[:, 0]
df['RACA'] = df[['RACA', 'raca']].bfill(axis=1).iloc[:, 0]
df['CODESC'] = df[['CODESC', 'codes']].bfill(axis=1).iloc[:, 0]
df['TIPOESC'] = df[['TIPOESC', 'tipoesc']].bfill(axis=1).iloc[:, 0]
df['DESCTURNO'] = df[['DESCTURNO', 'periodo']].bfill(axis=1).iloc[:, 0]
df['DATABASE'] = df[['DATABASE', 'database']].bfill(axis=1).iloc[:, 0]



columns_to_drop = ['qtd', 'dre', 'nomesc', 'distrito', 'setor', 'ano', 'rede', 'modal', 
                   'descserie', 'periodo', 'turno', 'sexo', 'idade', 'nee', 'raca', 'Qtde','MODALID', 'ï»¿dre', 'codes',
       'tipoesc', 'descturno', 'database','SERIEV','SETEDU']
df = df.drop(columns=columns_to_drop)

df['DRE'] = df['DRE'].str.strip()
df['RACA'] = df['RACA'].fillna('NAO DECLARADA')
df['PERIODO'] = df['PERIODO'].replace('Diurno','DIURNO')
df['PERIODO'] = df['PERIODO'].replace('Noturno','NOTURNO')
df['DESCTURNO'] = df['DESCTURNO'].replace('Tarde','TARDE')
df['DESCTURNO'] = df['DESCTURNO'].replace('Manhã','MANHA')
df['DESCTURNO'] = df['DESCTURNO'].replace('Integral','INTEGRAL')
df['DESCTURNO'] = df['DESCTURNO'].replace('Noite','NOITE')
df['DESCTURNO'] = df['DESCTURNO'].replace('Vespertino','VESPERTINO')
df['DESCTURNO'] = df['DESCTURNO'].replace('Intermediário','INTERMEDIARIO')
df['RACA'] = df['RACA'].replace('NÃO DECLARADA','NAO INFORMADA')
df['RACA'] = df['RACA'].replace('RECUSOU INFORMA','NAO INFORMADA')
df['RACA'] = df['RACA'].replace('RECUSOU INFORMAR','NAO INFORMADA')
df['DATABASE'] = df['DATABASE'].replace('30/12/2022 00:00','30/12/2022')
df['DATABASE'] = df['DATABASE'].replace('28-dez-17','28/12/2017')
df['DATABASE'] = df['DATABASE'].replace('25/12/2023 00:00','25/12/2023')






df.to_excel("arquivo_integrado.xlsx", index=False)

print("As colunas repetidas foram integradas com sucesso!")

## CRIAÇÃO DAS TABELAS NO SQL

In [ ]:
cursor.execute(f"CREATE TABLE IF NOT EXISTS escolas ({colunas_escolas})")
cursor.execute(f"CREATE TABLE IF NOT EXISTS educandos ({colunas_educandos})")


# Inserir os dados na tabela
print("escrevendo dados no df_escolas")
df_escolas.to_sql("escolas", conn, if_exists="replace", index=False)
print('escrevendo dados no df_educandos')
df_educandos.to_sql("educandos", conn, if_exists="replace", index=False)

# Fechar a conexão
conn.close()

print("Dados importados com sucesso!")

## CONSULTAS

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

def conectar_banco(db_name="educacao.db"):
    conexao = sqlite3.connect(db_name)
    cursor = conexao.cursor()
    return conexao, cursor

def query(sql, cursor):
    cursor.execute(sql)
    result = cursor.fetchall()
    return result

def criar_tabelas(conexao, cursor):
    cursor.execute('''CREATE TABLE IF NOT EXISTS escolas (
                        CODESC INTEGER PRIMARY KEY,
                        NOMESC TEXT,
                        Endereco TEXT,
                        Latitude REAL,
                        Longitude REAL,
                        CEP TEXT)''')
    
    cursor.execute('''CREATE TABLE IF NOT EXISTS educandos (
                        ID INTEGER PRIMARY KEY AUTOINCREMENT,
                        TURNO TEXT,
                        SEXO TEXT,
                        IDADE INTEGER,
                        NEE TEXT,
                        RACA TEXT,
                        CODESC INTEGER,
                        Distrito TEXT,
                        FOREIGN KEY (CODESC) REFERENCES escolas (CODESC))''')
    conexao.commit()

def gerar_graficos(df, titulo, tipo='bar'):
    plt.figure(figsize=(10, 5))
    if tipo == 'bar':
        df.plot(kind='bar', legend=False)
    elif tipo == 'pie':
        df.plot(kind='pie', autopct='%1.1f%%', legend=False)
    plt.title(titulo)
    plt.show()

## QUAL A PROPORÇÃO ENTRE ALUNOS PELA PERSPÉCTIVA DOS PERFIS.

In [ ]:
def proporcao_alunos_por_perfil(cursor):
    consultas = {
        "Turno": "SELECT TURNO, COUNT(*) * 100.0 / (SELECT COUNT(*) FROM educandos) FROM educandos GROUP BY TURNO",
        "Sexo": "SELECT SEXO, COUNT(*) * 100.0 / (SELECT COUNT(*) FROM educandos) FROM educandos GROUP BY SEXO",
        "Idade": "SELECT IDADE, COUNT(*) * 100.0 / (SELECT COUNT(*) FROM educandos) FROM educandos GROUP BY IDADE",
        "NEE": "SELECT NEE, COUNT(*) * 100.0 / (SELECT COUNT(*) FROM educandos) FROM educandos GROUP BY NEE",
        "Raça": "SELECT RACA, COUNT(*) * 100.0 / (SELECT COUNT(*) FROM educandos) FROM educandos GROUP BY RACA",
    }
    for key, sql in consultas.items():
        result = query(sql, cursor)
        df = pd.DataFrame(result, columns=[key, 'Proporcao']).set_index(key)
        gerar_graficos(df, f"Proporção de alunos por {key}", tipo='pie' if key in ['Sexo', 'Raça', 'NEE'] else 'bar')



## QUAL A DENSIDADE ENTRE AS PERFIS E ESCOLAS

In [ ]:
def densidade_alunos_por_escola(cursor):
    sql = '''SELECT e.NOMESC, COUNT(ed.ID) AS Total_Alunos
             FROM escolas e
             LEFT JOIN educandos ed ON e.CODESC = ed.CODESC
             GROUP BY e.NOMESC'''
    result = query(sql, cursor)
    df = pd.DataFrame(result, columns=['Escola', 'Total Alunos']).set_index('Escola')
    gerar_graficos(df, "Densidade de alunos por escola")

## QUAL A PROPOÇÃO DE ALUNOS POR DISTRITO

In [ ]:
def proporcao_alunos_por_distrito(cursor):
    sql = '''SELECT Distrito, COUNT(*) * 100.0 / (SELECT COUNT(*) FROM educandos) AS Proporcao
             FROM educandos
             GROUP BY Distrito'''
    result = query(sql, cursor)
    df = pd.DataFrame(result, columns=['Distrito', 'Proporcao']).set_index('Distrito')
    gerar_graficos(df, "Proporção de alunos por distrito")

def main():
    conexao, cursor = conectar_banco()
    criar_tabelas(conexao, cursor)
    
    proporcao_alunos_por_perfil(cursor)
    densidade_alunos_por_escola(cursor)
    proporcao_alunos_por_distrito(cursor)
    
    conexao.close()

if __name__ == "__main__":
    main()